In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from matplotlib import  pyplot as plt
import requests
import folium

%matplotlib inline

## Get Web Content
Use the requests package to get the web content, then use the beautifulsoup to parse the data

In [2]:
# get the page
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)

In [3]:
# load html
soup = BeautifulSoup(page.content, "lxml")

In [4]:
tablehead = soup.find("table").find_all("th")

In [5]:
# get the data
columns = []
for i in tablehead:
    columns.append(i.text.strip())

In [6]:
tabledata = soup.find("table").find_all("tr")[1:]

## Create Data
When parse the table content, collect the data into the dataframe.Finally, group by the postcode and Borough to paste the Neighbourhood together

In [7]:
# create the dataframe
data = pd.DataFrame(columns=columns)
index = 0
for i in tabledata:
    temp = []
    for item in i.find_all("td"):
        temp.append(item.text.strip())
    if "Not assigned" not in temp:
        data = pd.concat([data, pd.DataFrame(dict(zip(columns, temp)), index=[index])])
        index += 1

In [8]:
data = data.groupby(["Postcode", "Borough"])["Neighbourhood"].apply(lambda x: ", ".join(x.tolist())).reset_index()

In [9]:
data.loc[data.Postcode == "M5A"]

,Postcode,Borough,Neighbourhood
53,M5A,Downtown Toronto,"Harbourfront, Regent Park"


## Merge Geo Infor
Merge the geo information into the data

In [10]:
# load geo data
geodata = pd.read_csv("./Geospatial_Coordinates.csv")
geodata.rename({"Postal Code":"Postcode"}, axis=1, inplace=True)

In [11]:
# merge the geo data
data = data.merge(geodata, how="left", on="Postcode")

In [12]:
data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
